In [1]:
!pip install scikit-fuzzy

   ---------------------------------------- 0.0/920.8 kB ? eta -:--:--
   ----------- ---------------------------- 262.1/920.8 kB ? eta -:--:--
   ---------------------- ----------------- 524.3/920.8 kB 1.4 MB/s eta 0:00:01
   ---------------------------------- ----- 786.4/920.8 kB 1.4 MB/s eta 0:00:01
   ---------------------------------------- 920.8/920.8 kB 870.1 kB/s  0:00:00


In [ ]:
# ===========================================
# SISTEMA EXPERTO DIFUSO - DETECCIÓN DE TERREMOTOS
# Versión completa (sin zonas muertas)
# ===========================================

!pip install -q scikit-fuzzy

import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import matplotlib.pyplot as plt

# ---- Utilidad de entrada segura ----
def leer_float(prompt, minimo, maximo):
    """Lee un float dentro de [minimo, maximo] con reintentos."""
    while True:
        try:
            v = float(input(prompt))
            if v < minimo or v > maximo:
                print(f"Valor fuera de rango [{minimo}–{maximo}]. Intenta de nuevo.")
                continue
            return v
        except Exception:
            print("Entrada inválida. Ingresa un número válido.")

# --- Definición de variables ---
aceleracion = ctrl.Antecedent(np.arange(0, 1.1, 0.01), 'aceleracion')   # g (0–1)
frecuencia  = ctrl.Antecedent(np.arange(0, 20.1, 0.1), 'frecuencia')     # Hz (0–20)
duracion    = ctrl.Antecedent(np.arange(0, 60.1, 1), 'duracion')         # s (0–60)
snr         = ctrl.Antecedent(np.arange(0, 30.1, 0.1), 'snr')            # dB (0–30)
distancia   = ctrl.Antecedent(np.arange(0, 601, 1), 'distancia')        # km (0–600)
amenaza     = ctrl.Consequent(np.arange(0, 11, 1), 'amenaza')             # Nivel (0–10)

# --- Funciones de pertenencia ---
## Distancia (km)
distancia['muy_cercana'] = fuzz.trapmf(distancia.universe, [0, 0, 50, 150]) 
distancia['cercana'] = fuzz.trimf(distancia.universe, [100, 200, 300]) 
distancia['moderada'] = fuzz.trimf(distancia.universe, [250, 375, 500])
distancia['lejana'] = fuzz.trapmf(distancia.universe, [450, 550, 600, 600])

aceleracion['baja']  = fuzz.trimf(aceleracion.universe, [0, 0, 0.3])
aceleracion['media'] = fuzz.trimf(aceleracion.universe, [0.2, 0.5, 0.8])
aceleracion['alta']  = fuzz.trimf(aceleracion.universe, [0.6, 1, 1])

frecuencia['baja']   = fuzz.trapmf(frecuencia.universe, [0, 0, 2, 5])
frecuencia['media']  = fuzz.trapmf(frecuencia.universe, [3, 6, 10, 13])
frecuencia['alta']   = fuzz.trapmf(frecuencia.universe, [11, 15, 20, 20])

duracion['corta']    = fuzz.trimf(duracion.universe, [0, 0, 10])
duracion['media']    = fuzz.trimf(duracion.universe, [5, 20, 35])
duracion['larga']    = fuzz.trimf(duracion.universe, [30, 60, 60])

snr['pobre']         = fuzz.trapmf(snr.universe, [0.0, 0.0, 4.0, 6.0])
snr['marginal']      = fuzz.trimf( snr.universe, [6.0, 7.75, 9.5])
snr['operacional']   = fuzz.trimf( snr.universe, [9.5, 12.0, 14.0])
snr['buena']         = fuzz.trimf( snr.universe, [14.0, 17.0, 20.0])
snr['excelente']     = fuzz.trapmf(snr.universe, [20.0, 23.0, 30.0, 30.0])

amenaza['sin_actividad'] = fuzz.trimf(amenaza.universe, [0, 0, 2])
amenaza['microtemblor']  = fuzz.trimf(amenaza.universe, [1, 3, 5])
amenaza['leve']          = fuzz.trimf(amenaza.universe, [4, 5, 6])
amenaza['moderado']      = fuzz.trimf(amenaza.universe, [5, 7, 8])
amenaza['fuerte']        = fuzz.trimf(amenaza.universe, [7, 9, 10])
amenaza['destructivo']   = fuzz.trimf(amenaza.universe, [9, 10, 10])

# --- Reglas completas (3x3x3x5 = 135 combinaciones) ---
rules = []

niveles_acel = ['baja', 'media', 'alta']
niveles_freq = ['baja', 'media', 'alta']
niveles_dur  = ['corta', 'media', 'larga']
niveles_snr  = ['pobre', 'marginal', 'operacional', 'buena', 'excelente']
niveles_dist = ['muy_cercana', 'cercana', 'moderada', 'lejana']

# Orden y mapas para ajustar por SNR
orden_salidas = ['sin_actividad','microtemblor','leve','moderado','fuerte','destructivo']
idx_salida    = {name:i for i, name in enumerate(orden_salidas)}
ajuste_snr    = {'pobre':-1, 'marginal':0, 'operacional':0, 'buena':+1, 'excelente':+1}

def base_output_por(a, f, d):
    # Lógica original
    if a == 'baja' and f == 'alta' and d == 'corta':
        return 'sin_actividad'
    elif a == 'baja':
        return 'microtemblor'
    elif a == 'media' and f == 'alta':
        return 'leve'
    elif a == 'media' and f == 'media' and d != 'corta':
        return 'moderado'
    elif a == 'media':
        return 'leve'
    elif a == 'alta' and f == 'alta' and d == 'corta':
        return 'microtemblor'
    elif a == 'alta' and f == 'media' and d == 'corta':
        return 'leve'
    elif a == 'alta' and f == 'baja' and d == 'larga':
        return 'destructivo'
    elif a == 'alta' and f == 'baja':
        return 'fuerte'
    else:
        return 'moderado'

for a in niveles_acel:
    for f in niveles_freq:
        for d in niveles_dur:
            base = base_output_por(a, f, d)
            base_idx = idx_salida[base]
            for s in niveles_snr:
                # Ajuste por calidad de señal (SNR)
                nuevo_idx = max(0, min(len(orden_salidas)-1, base_idx + ajuste_snr[s]))
                out = orden_salidas[nuevo_idx]
                rules.append(ctrl.Rule(aceleracion[a] & frecuencia[f] & duracion[d] & snr[s], amenaza[out]))

# Crear sistema de control
control_sismo = ctrl.ControlSystem(rules)
simulador = ctrl.ControlSystemSimulation(control_sismo)

# --- Entrada por terminal ---
try:
    valor_acel = leer_float("Ingrese la aceleración del terreno (g, 0–1): ", 0.0, 1.0)
    valor_freq = leer_float("Ingrese la frecuencia dominante (Hz, 0–20): ", 0.0, 20.0)
    valor_dur  = leer_float("Ingrese la duración del evento (s, 0–60): ", 0.0, 60.0)
    valor_dist = leer_float("Ingrese la distancia a borde de placa tectónica (km, 0–600): ", 0.0, 600.0)
    valor_snr  = leer_float("Ingrese el SNR (dB, 0–30): ", 0.0, 30.0)
except Exception:
    print("❌ Error de entrada. Reinicia la celda e ingresa valores numéricos válidos.")
    raise SystemExit(1)

# Asignar y calcular
simulador.input['aceleracion'] = valor_acel
simulador.input['frecuencia']  = valor_freq
simulador.input['duracion']    = valor_dur
simulador.input['distancia']   = valor_dist
simulador.input['snr']         = valor_snr
simulador.compute()

# --- Mostrar resultados ---
nivel_amenaza = simulador.output['amenaza']
print(f"\n🔍 Nivel de amenaza sísmica (defuzzificado): {nivel_amenaza:.2f} / 10")

if nivel_amenaza < 2:
    print("➡️ Clasificación: Sin actividad sísmica.")
elif nivel_amenaza < 4:
    print("➡️ Clasificación: Microtemblor.")
elif nivel_amenaza < 6:
    print("➡️ Clasificación: Terremoto leve.")
elif nivel_amenaza < 8:
    print("➡️ Clasificación: Terremoto moderado.")
elif nivel_amenaza < 9.5:
    print("➡️ Clasificación: Terremoto fuerte.")
else:
    print("➡️ Clasificación: Terremoto destructivo.")

# --- Gráficas ---
# Nota: cada .view() abre su propia figura; se mantiene el patrón original.
# aceleracion.view()
# frecuencia.view()
# duracion.view()
# snr.view()
# amenaza.view(simulador)
# plt.show()



🔍 Nivel de amenaza sísmica: 0.67 / 10
